# Chaper.19 GAN

In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Reshape,Conv2D,Dropout,Flatten,BatchNormalization,Activation,Input,UpSampling2D,LeakyReLU
from tensorflow.keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
generator = Sequential()
generator.add(Dense(7*7*128, input_dim=100, activation=LeakyReLU(.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D())
generator.add(Conv2D(128, kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1, kernel_size=5, padding='same',activation='tanh'))

# generator.summary()

In [3]:
discriminator = Sequential()
discriminator.add(Conv2D(64,kernel_size =5, padding='same',strides=2, input_shape=(28,28,1)))  # padding을 해도 strudes=2 에서 차원이 반으로 줄어든다
discriminator.add(Activation(LeakyReLU(alpha=0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128,kernel_size =5, padding='same',strides=2))     # padding을 해도 strudes=2 에서 차원이 반으로 줄어든다
discriminator.add(Activation(LeakyReLU(alpha=0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
discriminator.trainable = False

discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation_1 (Activation)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_2 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [4]:
ginput = Input(shape=(100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         1072001   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,284,866
Trainable params: 1,059,201
Non-trainable params: 225,665
_________________________________________________________________


In [6]:
def gan_train(epochs, batch_size,saving_interval):
    (X_train,_), (_, _) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28,28,1).astype('float32')
    X_train = (X_train - 127.5) / 127.5
    
    true = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))
    
    for i in range(epochs):
        idx = np.random.randint(0,X_train.shape[0],batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs,true)
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs,fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise,true)
        print('epoch : {}, d_loss : {}, g_loss : {}'.format(i, d_loss, g_loss))
        
        if i % saving_interval == 0:
            noise = np.random.normal(0, 1, (25, 100))
            gen_imgs = generator.predict(noise)
            gen_imgs = 0.5 * gen_imgs + 0.5
            
            fig, axs = plt.subplots(5,5)
            count = 0
            for j in range(5):
                for k in range(5):
                    axs[j,k].imshow(gen_imgs[count, :,:,0], cmap='gray')
                    axs[j,k].axis('off')
                    count += 1
            fig.savefig('data/gan_images/gan_mnist_{}.png'.format(i))


In [8]:
# 위 함수 실행
gan_train(5001,200,500)

7/7 [==============================] - 0s 65ms/step
epoch : 0, d_loss : [0.30609637 0.86999997], g_loss : 2.288701057434082
7/7 [==============================] - 1s 68ms/step
epoch : 1, d_loss : [0.23700047 0.91250002], g_loss : 2.8254289627075195
7/7 [==============================] - 1s 70ms/step
epoch : 2, d_loss : [0.33858798 0.86000001], g_loss : 2.3783206939697266
7/7 [==============================] - 1s 68ms/step
epoch : 3, d_loss : [0.27938172 0.86499998], g_loss : 1.8767733573913574
7/7 [==============================] - 0s 65ms/step
epoch : 4, d_loss : [0.26113821 0.88749999], g_loss : 1.8143943548202515
7/7 [==============================] - 0s 49ms/step
epoch : 5, d_loss : [0.23588274 0.91249999], g_loss : 2.3747384548187256
7/7 [==============================] - 0s 51ms/step
epoch : 6, d_loss : [0.21467425 0.91      ], g_loss : 3.0155134201049805
7/7 [==============================] - 0s 56ms/step
epoch : 7, d_loss : [0.24470677 0.88750002], g_loss : 3.0805771350860596
7